In [1]:
!pip install catboost

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier,Pool,cv

In [2]:
df = pd.read_csv("train.gz")

In [3]:
df = df[:1_000_000]

In [4]:
merge_df = pd.read_parquet('/notebooks/als_features.parquet.gzip').rename({'user_id':'device_ip'},axis=1)

In [5]:
merge_df

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_119,factor_120,factor_121,factor_122,factor_123,factor_124,factor_125,factor_126,factor_127,device_ip
0,12.250009,26.496704,30.829353,-1.798806,22.144594,4.328948,2.126080,-17.942051,-3.645249,16.645702,...,22.079058,-13.334606,25.372416,5.133678,-20.894043,22.699482,-1.860252,-0.706228,8.472021,ddd2926e
1,-0.030936,-0.002875,0.071858,-0.021331,0.030414,-0.005243,0.008320,0.066662,-0.088213,0.066484,...,-0.067610,0.054570,-0.036901,0.057322,0.074313,0.041369,0.090696,-0.103753,0.079967,96809ac8
2,-0.030745,-0.002856,0.071421,-0.021216,0.030237,-0.005210,0.008273,0.066258,-0.087667,0.066086,...,-0.067204,0.054239,-0.036684,0.056969,0.073867,0.041122,0.090152,-0.103126,0.079483,b3cf8def
3,0.273354,0.124838,-0.132490,0.195788,-0.411586,0.280071,0.183847,0.004332,0.016017,0.166078,...,0.290289,-0.133091,-0.417909,0.481402,-0.302184,0.439968,0.070422,0.403348,0.502460,e8275b8f
4,-0.142969,-0.027345,0.045131,0.203992,-0.000520,0.155052,-0.054078,-0.075375,-0.050301,0.079220,...,0.156836,0.111901,-0.089880,-0.041745,0.103389,0.116090,-0.080860,-0.451459,0.014922,9644d0bf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6729481,-0.053585,-0.026912,0.070562,0.047164,0.006380,0.089126,0.073524,0.028141,0.004007,0.028699,...,0.025827,-0.065654,-0.043786,0.039549,0.026819,-0.029950,0.019904,-0.046702,0.043549,ca8e882a
6729482,-0.058341,-0.119619,0.026297,0.038807,0.009493,-0.007967,-0.006365,0.057529,-0.024215,-0.021118,...,-0.018249,0.097975,0.026639,0.089570,-0.134738,-0.049780,-0.031872,-0.047899,0.048068,ecc24bba
6729483,-0.045810,0.006729,0.076871,0.056743,0.066720,0.047379,0.018953,-0.067489,0.177071,0.032407,...,-0.191796,-0.034866,0.153290,0.011871,-0.064005,-0.094216,0.072364,-0.037351,0.055132,d9e2a457
6729484,-0.053585,-0.026912,0.070562,0.047164,0.006380,0.089126,0.073524,0.028141,0.004007,0.028699,...,0.025827,-0.065654,-0.043786,0.039549,0.026819,-0.029950,0.019904,-0.046702,0.043549,7c9e21ff


In [6]:
df = df.merge(merge_df,on='device_ip',how='inner')

In [7]:
def get_features(df_train,df_test,cols):
    for col in tqdm(cols):
        group = df_train[['click',col]].groupby(col)#.agg(['mean','count'])
        agg_group = group.agg(['mean','count'])
        
        agg_group[col] = agg_group.index
        agg_group[f'{col}_mean'] = agg_group['click']['mean']
        agg_group[f'{col}_count'] = agg_group['click']['count']
        agg_group = agg_group.drop(['click'],axis=1)
        agg_group = agg_group.set_axis([col, f'{col}_mean', f'{col}_count'], axis=1)
        agg_group.index = agg_group.index.rename('index')
        
        df_train[f'{col}_mean'] = group.transform('mean')
        df_train[f'{col}_count'] = group.transform('count')
        
        df_test = df_test.merge(agg_group,how='inner')
    return df_train,df_test

In [8]:
cat_features = ['C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type',
        'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
feature_cols = ['site_id','app_id','device_ip','C14','device_model','device_id','site_domain',
                'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
num_features = [f'factor_{i}' for i in range(128)]#['month', 'day','week', 'quarter', 'weekofyear', 'dayofweek',
               # 'dayofyear', 'all_time','all_week']
features = cat_features + num_features
label_col = 'click'

In [10]:
#train_df = df[:800_000]
#val_df = df[800_000:]
train_df,val_df = train_test_split(df,test_size=0.2,random_state=56)

In [11]:
#del df

In [12]:
features

['C1',
 'banner_pos',
 'site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model',
 'device_type',
 'device_conn_type',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'factor_0',
 'factor_1',
 'factor_2',
 'factor_3',
 'factor_4',
 'factor_5',
 'factor_6',
 'factor_7',
 'factor_8',
 'factor_9',
 'factor_10',
 'factor_11',
 'factor_12',
 'factor_13',
 'factor_14',
 'factor_15',
 'factor_16',
 'factor_17',
 'factor_18',
 'factor_19',
 'factor_20',
 'factor_21',
 'factor_22',
 'factor_23',
 'factor_24',
 'factor_25',
 'factor_26',
 'factor_27',
 'factor_28',
 'factor_29',
 'factor_30',
 'factor_31',
 'factor_32',
 'factor_33',
 'factor_34',
 'factor_35',
 'factor_36',
 'factor_37',
 'factor_38',
 'factor_39',
 'factor_40',
 'factor_41',
 'factor_42',
 'factor_43',
 'factor_44',
 'factor_45',
 'factor_46',
 'factor_47',
 'factor_48',
 'factor_49',
 'factor_50',
 'factor_51',
 'factor_52',
 'factor_53

In [13]:
train_pool = Pool(train_df[features],
                  label=train_df[label_col],
                  cat_features=cat_features)

eval_pool = Pool(val_df[features],
                  label=val_df[label_col],
                  cat_features=cat_features)

In [14]:
params = {'iterations':2_000,
          'learning_rate':0.1,
          'loss_function':'CrossEntropy',
          'max_depth':7,
          'eval_metric':'AUC',
          'task_type':'GPU',
          'leaf_estimation_method':'Newton',
          'gpu_ram_part':0.8,
          'one_hot_max_size':32,
          'random_seed':56}
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_pool,eval_set=eval_pool,verbose=1)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7183849	best: 0.7183849 (0)	total: 44.7ms	remaining: 1m 29s
1:	total: 84.3ms	remaining: 1m 24s
2:	total: 123ms	remaining: 1m 22s
3:	total: 164ms	remaining: 1m 22s
4:	total: 202ms	remaining: 1m 20s
5:	test: 0.7424757	best: 0.7424757 (5)	total: 242ms	remaining: 1m 20s
6:	total: 285ms	remaining: 1m 21s
7:	total: 329ms	remaining: 1m 21s
8:	total: 368ms	remaining: 1m 21s
9:	total: 413ms	remaining: 1m 22s
10:	test: 0.7499391	best: 0.7499391 (10)	total: 461ms	remaining: 1m 23s
11:	total: 495ms	remaining: 1m 22s
12:	total: 538ms	remaining: 1m 22s
13:	total: 585ms	remaining: 1m 22s
14:	total: 629ms	remaining: 1m 23s
15:	test: 0.7611204	best: 0.7611204 (15)	total: 675ms	remaining: 1m 23s
16:	total: 715ms	remaining: 1m 23s
17:	total: 756ms	remaining: 1m 23s
18:	total: 797ms	remaining: 1m 23s
19:	total: 842ms	remaining: 1m 23s
20:	test: 0.7743983	best: 0.7743983 (20)	total: 883ms	remaining: 1m 23s
21:	total: 927ms	remaining: 1m 23s
22:	total: 965ms	remaining: 1m 22s
23:	total: 1.01s	rem

0.6888682246


In [25]:
preds = model.predict_proba(eval_pool)[:,1]

In [27]:
from sklearn.metrics import roc_auc_score

In [29]:
roc_auc_score(val_df[label_col],preds)

0.810079574859153

In [41]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,app_id,18.001241
1,site_id,15.552773
2,device_ip,14.183223
3,site_domain,8.822575
4,device_model,8.410217
5,C14,7.657343
6,device_id,5.039237
7,C17,4.853359
8,C21,3.243395
9,site_category,2.463928


In [30]:
model.get_feature_importance(prettified=True)[:30]

,Feature Id,Importances
0,device_ip,17.041649
1,app_id,14.430748
2,site_id,13.558149
3,device_model,9.372574
4,site_domain,6.256414
5,C17,5.447896
6,C14,4.592051
7,C21,4.292141
8,device_id,2.496033
9,factor_75,1.800913
